In [ ]:
import capblood_seq
import scvi
from scvi.dataset import GeneExpressionDataset
import numpy
from sparsedat import wrappers 
import scipy
from scvi.models.vae import VAE
from scvi.inference import UnsupervisedTrainer
from scvi.models.scanvi import SCANVI
import scanpy as sc
from plotly import offline as plotly
from sklearn.manifold import TSNE
from plotly import graph_objects
from umap import UMAP
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import torch
import pickle
import os
import random
import pandas as pd
from scvi import set_seed
import pickle
from scrapi.dataset import Gene_Expression_Dataset as GED
from capblood_seq import config
import pandas

In [ ]:
SEED=1040

In [ ]:
capblood_seq_data = capblood_seq.load_dataset("data", pipeline_name="debris_filtered")
# Combine transcript counts from the different samples into one big matrix

cell_sample_index = []
combined_transcript_counts = []

for sample_index,sample in enumerate(config.SAMPLE_NAMES):
    ged = capblood_seq_data._sample_datasets[sample]
    cell_transcript_counts = ged.get_cell_transcript_counts()
    cell_transcript_counts = cell_transcript_counts.to_array()
    combined_transcript_counts.append(cell_transcript_counts)
    cell_sample_index.extend([sample_index]*cell_transcript_counts.shape[0])

combined_transcript_counts = numpy.concatenate(
   combined_transcript_counts, axis=0)

In [ ]:
#Populate the gene expression set with data
ged = GeneExpressionDataset()

ged.populate_from_data(
    combined_transcript_counts,
    gene_names=capblood_seq_data.gene_list,
    batch_indices=cell_sample_index
)

In [ ]:
# Initiliaze variationan autoencoder and training parameters
n_epochs = 50
learning_rate = 1e-3
num_clusters = 13
# Save the training weights
latent_pickle_file_name = os.path.join("data", "Dobreva2020", "dobreva2020_nepoch_%i_lr_%.1e_latent.pickle" % (n_epochs, learning_rate))
weights_pickle_file_name = os.path.join("data", "Dobreva2020", "dobreva2020_nepoch_%i_lr_%.1e_weights.pickle" % (n_epochs, learning_rate))

In [ ]:
vae = VAE(ged.nb_genes,n_batch=ged.n_batches)
trainer=UnsupervisedTrainer(vae,ged,train_size=0.8,frequency=1,seed=SEED)

In [ ]:
# If you can't load existing latent space, train!
if not os.path.exists(latent_pickle_file_name):

    set_seed(SEED)
    
    trainer.train(n_epochs=n_epochs, lr=learning_rate)
    torch.save(trainer.model.state_dict(), weights_pickle_file_name)
    
    full = trainer.create_posterior(trainer.model, ged, indices=numpy.arange(len(ged)))
    latent, _, _ = full.sequential().get_latent()
    
    with open(latent_pickle_file_name, 'wb') as latent_pickle_file:
        pickle.dump(latent, latent_pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

else:
    
    weights_pickle_file = torch.load(weights_pickle_file_name)
    trainer.model.load_state_dict(weights_pickle_file)
    
    full = trainer.create_posterior(trainer.model, ged, indices=numpy.arange(len(ged)))
    
    with open(latent_pickle_file_name, 'rb') as latent_pickle_file:
        latent = pickle.load(latent_pickle_file)

In [ ]:
tsne = TSNE(n_components = 2,random_state=SEED).fit_transform(latent)

In [ ]:
clusters = AgglomerativeClustering(n_clusters=num_clusters).fit_predict(latent)

In [ ]:
traces = []

for cluster_index in range(clusters.max()+1):
    
    x = tsne[clusters == cluster_index, 0]
    y = tsne[clusters == cluster_index, 1]
    
    trace = graph_objects.Scatter(
        x=x,
        y=y,
        name="Cluster %i" % cluster_index,
        mode="markers"
    )
    
    traces.append(trace)

figure = graph_objects.Figure(traces)

plotly.iplot(figure)

In [ ]:
traces = []
    
x = tsne[:, 0]
y = tsne[:, 1]

trace = graph_objects.Scatter(
    x=x,
    y=y,
    name="Cluster %i" % cluster_index,
    mode="markers",
    marker={
        "color": numpy.array(combined_transcript_counts.sum(axis=1)).flatten()
    },
    text=numpy.array(combined_transcript_counts.sum(axis=1)).flatten()
)

traces.append(trace)

figure = graph_objects.Figure(traces)

plotly.iplot(figure)

In [ ]:
traces = []
    
x = tsne[:, 0]
y = tsne[:, 1]

mt_ratio = numpy.array(combined_transcript_counts[:, numpy.char.startswith(capblood_seq_data.gene_list, "MT-")].sum(axis=1)).flatten()
mt_ratio = mt_ratio/numpy.array(combined_transcript_counts.sum(axis=1)).flatten()

trace = graph_objects.Scatter(
    x=x,
    y=y,
    name="Cluster %i" % cluster_index,
    mode="markers",
    marker={
        "color": mt_ratio
    },
    text=mt_ratio
)

traces.append(trace)

figure = graph_objects.Figure(traces)

plotly.iplot(figure)

In [ ]:
GENE = "CD14"

gene_index = capblood_seq_data.gene_list.index(GENE)

traces = []
    
x = tsne[:, 0]
y = tsne[:, 1]

trace = graph_objects.Scatter(
    x=x,
    y=y,
    name="Cluster %i" % cluster_index,
    mode="markers",
    marker={
        "color": combined_transcript_counts[:, gene_index].flatten()

    })

traces.append(trace)

figure = graph_objects.Figure(traces)

plotly.iplot(figure)

In [ ]:
cluster_cell_marker_map = {}
cluster_cell_marker_map[2] = 'CD4 T Cells'
cluster_cell_marker_map[6] = 'CD4 T Cells'

cluster_cell_marker_map[3] = 'CD8 T Cells'
cluster_cell_marker_map[9] = 'CD8 T Cells'

cluster_cell_marker_map[7] = 'NK Cells'
cluster_cell_marker_map[10] = 'NK Cells'

cluster_cell_marker_map[11] = 'B Cells'
cluster_cell_marker_map[12] = 'B Cells'

cluster_cell_marker_map[0] = 'CD14 Monocytes'

cluster_cell_marker_map[8] = 'CD16 Monocytes'

# cluster_cell_marker_map[1] = 'Unknown'
# cluster_cell_marker_map[5] = 'Unknown'

In [ ]:
num_cells = 0

for cell_type in set(cluster_cell_marker_map.values()):
    
    cell_type_cells = clusters == -1
    
    for cluster in cluster_cell_marker_map:
        
        if cluster_cell_marker_map[cluster] == cell_type:
            cell_type_cells = cell_type_cells | (clusters == cluster)
    
    num_cell_type_cells = cell_type_cells.sum()
    
    num_cells += num_cell_type_cells

print(num_cells)

In [ ]:
traces = []

for cell_type in set(cluster_cell_marker_map.values()):
    
    cell_type_cells = clusters == -1
    
    for cluster in cluster_cell_marker_map:
        
        if cluster_cell_marker_map[cluster] == cell_type:
            cell_type_cells = cell_type_cells | (clusters == cluster)
    
    x = tsne[cell_type_cells, 0]
    y = tsne[cell_type_cells, 1]
    
    trace = graph_objects.Scatter(
        x=x,
        y=y,
        name="%s" % cell_type,
        mode="markers"
    )
    
    traces.append(trace)
    
# Blank the background
layout = graph_objects.Layout(
    {
        "plot_bgcolor": "rgba(255, 255, 255, 0)",
        "paper_bgcolor": "rgba(255, 255, 255, 0)"
    }
)

figure = graph_objects.Figure(traces, layout=layout)

plotly.iplot(figure)

# Save for publication!
figure.write_image(os.path.join("figures", "combined_tSNE_labeled_by_cell_type.svg"))
figure.write_html(os.path.join("figures", "combined_tSNE_labeled_by_cell_type.html"))

In [ ]:
excel_file = pandas.ExcelWriter(os.path.join("data", "Dobreva2020", "differential_expression.xlsx"), engine="xlsxwriter")

for cluster, cell_type in sorted(cluster_cell_marker_map.items(), key=lambda x: x[0]):

    print(cluster)
    
    cluster_cells = clusters == cluster
    non_cluster_cells = ~cluster_cells
    
    de_df = full.differential_expression_score(
        cluster_cells,
        non_cluster_cells,
        mode="change"
    )
    
    output_df = pandas.DataFrame(index=de_df.index, columns=["Bayes Factor", "Probability of DE", "Cluster Mean", "Non Cluster Mean"])
    
    output_df["Bayes Factor"] = de_df["bayes_factor"]
    output_df["Probability of DE"] = de_df["proba_de"]
    output_df["Cluster Mean"] = de_df["raw_mean1"]
    output_df["Non Cluster Mean"] = de_df["raw_mean2"]
    
    output_df = output_df.sort_values(by="Bayes Factor", key=lambda x: numpy.abs(x), ascending=False)
    
    output_df.to_excel(excel_file, sheet_name="Cluster %i - %s" % (cluster, cell_type))
    
for cell_type in set(cluster_cell_marker_map.values()):
    
    print(cell_type)
    
    cell_type_cells = clusters == -1
    
    for cluster in cluster_cell_marker_map:
        
        if cluster_cell_marker_map[cluster] == cell_type:
            cell_type_cells = cell_type_cells | (clusters == cluster)
            
    non_cell_type_cells = ~cell_type_cells
    
    de_df = full.differential_expression_score(
        cell_type_cells,
        non_cell_type_cells,
        mode="change"
    )
    
    output_df = pandas.DataFrame(index=de_df.index, columns=["Bayes Factor", "Probability of DE", "Cluster Mean", "Non Cluster Mean"])
    
    output_df["Bayes Factor"] = de_df["bayes_factor"]
    output_df["Probability of DE"] = de_df["proba_de"]
    output_df["Cluster Mean"] = de_df["raw_mean1"]
    output_df["Non Cluster Mean"] = de_df["raw_mean2"]
    
    output_df = output_df.sort_values(by="Bayes Factor", ascending=False)
    
    output_df.to_excel(excel_file, sheet_name="%s" % (cell_type))

excel_file.save()

In [ ]:
cell_subtype_labels = {}

for cell_type in config.CELL_TYPES:
    
    if cell_type in config.CELL_SUBTYPES:

        for cell_subtype in config.CELL_SUBTYPES[cell_type]:

            cell_subtype_labels["%s %s" % (cell_subtype, cell_type)] = cell_type

In [ ]:
cell_index = 0

for sample_index, sample in enumerate(config.SAMPLE_NAMES):
    
    ged = capblood_seq_data._sample_datasets[sample]
    
    num_cells = ged.num_cells
    
    # Rows are cells
    cell_barcodes = numpy.array(ged._cell_transcript_counts.row_names)
    
    for cluster, label in cluster_cell_marker_map.items():
        
        if label in ged.get_labels():
            ged.delete_label(label)
    
    for cluster, label in cluster_cell_marker_map.items():
        
        cluster_mask = clusters == cluster
        cluster_mask = cluster_mask[cell_index:cell_index+num_cells]
        
        label_barcodes = cell_barcodes[cluster_mask]
        label = cluster_cell_marker_map[cluster]
        
        ged.label_cells(label, label_barcodes)
        
        if label in cell_subtype_labels:        
            ged.label_cells(cell_subtype_labels[label], label_barcodes)
        
    ged.save_labels()
    
    cell_index += num_cells